# Model category 6: All available data/features
The last model category will use all avaialbe features.

## Model category specific configuration
These parameters are model category specific


In [1]:
# Model category name used throughout the subsequent analysis
model_cat_id = "06"

# Which features from the dataset should be loaded:
# ['all', 'actual', 'entsoe', 'weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month']
# features = ['all']
# features = ['actual', 'weather','calendar']
features = ['actual', 'weather','calendar']
# LSTM Layer configuration
# ========================

# Stateful True or false

layer_conf = [ True, True, True ]
# Number of neurons per layer


# cells = [[ 5, 10, 20, 30, 50, 75, 100, 125, 150 ], [0, 10, 20, 50], [0, 10, 15, 20]]
cells = [[ 1, 5, 10, 15 ], [0, 5, 10], [0, 5, 10]]
# Regularization per layer
dropout = [0, 0.1, 0.2]
# Size of how many samples are used for one forward/backward pass
batch_size = [8]
# In a sense this is the output neuron dimension, or how many timesteps the neuron should output. Currently not implemented, defaults to 1.
timesteps = [1]


### Module imports

In [2]:
import os
import sys
import math
import itertools
import datetime as dt
import pytz
import time as t
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import datetime
from numpy import newaxis

import matplotlib as mpl

import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa import stattools
from tabulate import tabulate

import math
import keras as keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.models import load_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from IPython.display import HTML
from IPython.display import display
%matplotlib notebook
mpl.rcParams['figure.figsize'] = (9,5)

# Import custom module functions
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lstm_load_forecasting import data, lstm

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


## Overall configuration
These parameters are later used, but shouldn't have to change between different model categories (model 1-5)

In [4]:
# Directory with dataset
path = os.path.join(os.path.abspath(''), 'E:/seminar project/input file for lstm neural netwrok/myData.csv')
# path = os.path.join(os.path.abspath(''), 'E:/seminar project/input file for lstm neural netwrok/fulldataset.csv')
# Splitdate for train and test data. As the TBATS and ARIMA benchmark needs 2 full cycle of all seasonality, needs to be after jan 01. 
# loc_tz = pytz.timezone('Europe/Zurich')
loc_tz = pytz.timezone('America/New_York')
# split_date = loc_tz.localize(dt.datetime(2004,10,1,0,0,0,0))
# split_date = loc_tz.localize(dt.datetime(2017,2,1,0,0,0,0))

# Validation split percentage
validation_split = 0.2
# How many epochs in total
epochs = 30
# Set verbosity level. 0 for only per model, 1 for progress bar...
verbose = 0

# Dataframe containing the relevant data from training of all models
results = pd.DataFrame(columns=['model_name', 'config', 'dropout',
                                'train_loss', 'train_rmse', 'train_mae', 'train_mape', 
                                'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape', 
                                'test_rmse', 'test_mae', 'test_mape',
                                'epochs', 'batch_train', 'input_shape',
                                'total_time', 'time_step', 'splits'
                               ])
# Early stopping parameters
early_stopping = True
min_delta = 0.006
patience = 2

## Preparation and model generation
Necessary preliminary steps and then the generation of all possible models based on the settings at the top of this notebook.

In [5]:
# Generate output folders and files
res_dir = 'E:/seminar project/lstm-load-forecasting/lstm-load-forecasting-master/results/notebook_' + model_cat_id + '/'
plot_dir = 'E:/seminar project/lstm-load-forecasting/lstm-load-forecasting-master/plots/notebook_' + model_cat_id + '/'
model_dir = 'E:/seminar project/lstm-load-forecasting/lstm-load-forecasting-master/models/notebook_' + model_cat_id + '/'
os.makedirs(res_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
output_table = res_dir + model_cat_id + '_results_' + t.strftime("%Y%m%d") + '.csv'
test_output_table = res_dir + model_cat_id + '_test_results' + t.strftime("%Y%m%d") + '.csv'

# Generate model combinations
models = []
models = lstm.generate_combinations(
    model_name=model_cat_id + '_', layer_conf=layer_conf, cells=cells, dropout=dropout, 
    batch_size=batch_size, timesteps=[1])

| Number of model configs generated | 108 |


## Loading the data:


In [6]:

#methdo 1
# df = data.load_dataset(path=path, modules=features)
# datatrain=df.loc[(df.index < split_date )].copy()
# datatest=df.loc[(df.index >= split_date)].copy()
# datatrain=datatrain.copy()
# datatrain=datatrain.dropna()
# datatest=datatest.copy()
# datatest=datatest.dropna()
# Xtrain = datatrain.drop('demand_load', 1).copy()
# ytrain = datatrain.drop('temperature', 1).copy()
# Xtest=datatest.drop('demand_load', 1).copy()
# ytest=datatest.drop('temperature', 1).copy()
# floatstrainx = [key for key in dict(Xtrain.dtypes) if dict(Xtrain.dtypes)[key] in ['float64']]
# floatstrainy = [key for key in dict(ytrain.dtypes) if dict(ytrain.dtypes)[key] in ['float64']]
# floatstestx = [key for key in dict(Xtest.dtypes) if dict(Xtest.dtypes)[key] in ['float64']]
# floatstesty = [key for key in dict(ytest.dtypes) if dict(ytest.dtypes)[key] in ['float64']]
# scalerx=StandardScaler()
# X_train=scalerx.fit_transform(Xtrain[floatstrainx])
# X_test=scalerx.fit_transform(Xtest[floatstestx])
# scalery=StandardScaler()
# y_train=scalery.fit_transform(ytrain[floatstrainy])
# y_test=scalery.fit_transform(ytest[floatstesty])
# print(y_train)


#method2
from sklearn.model_selection import train_test_split

A=['temperature','holiday', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
             'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
             'month_9', 'month_10', 'month_11', 'month_12']
B=['demand_load', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
             'month_9', 'month_10', 'month_11', 'month_12']
df = data.load_dataset(path=path, modules=features)
data_scaled=df.copy()
data_scaled=df.dropna()
X_data=data_scaled.drop(B,1).copy()
y_data=data_scaled.drop(A,1).copy()
# y_data=data_scaled.drop('temperature',1).copy()
floatsX=[key for key in dict(X_data.dtypes) if dict(X_data.dtypes)[key] in ['float64']]
floatsy=[key for key in dict(y_data.dtypes) if dict(y_data.dtypes)[key] in ['float64']]
scalerx=StandardScaler()
Xscaled=scalerx.fit_transform(X_data[floatsX])
X_data[floatsX]=Xscaled
scalery=StandardScaler()
yscaled=scalery.fit_transform(y_data[floatsy])
y_data[floatsy]=yscaled
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)
# print(X_test)



## Running through all generated models
Note: Depending on the above settings, this can take very long!

In [8]:
start_time = t.time()
for idx, m in enumerate(models):
    stopper = t.time()
    print('========================= Model {}/{} ========================='.format(idx+1, len(models)))
    print(tabulate([['Starting with model', m['name']], ['Starting time', datetime.fromtimestamp(stopper)]],
                   tablefmt="jira", numalign="right", floatfmt=".3f"))
    try:
        # Creating the Keras Model
        model = lstm.create_model(layers=m['layers'], sample_size=X_train.shape[0], batch_size=m['batch_size'], 
                          timesteps=m['timesteps'], features=X_train.shape[1])
        # Training...
        history = lstm.train_model(model=model, mode='fit', y=y_train, X=X_train, 
                                   batch_size=m['batch_size'], timesteps=m['timesteps'], epochs=epochs, 
                                   rearrange=False, validation_split=validation_split, verbose=verbose, 
                                   early_stopping=early_stopping, min_delta=min_delta, patience=patience)

        # Write results
        min_loss = np.min(history.history['val_loss'])
        min_idx = np.argmin(history.history['val_loss'])
        min_epoch = min_idx + 1

        if verbose > 0:
            print('______________________________________________________________________')
            print(tabulate([['Minimum validation loss at epoch', min_epoch, 'Time: {}'.format(t.time()-stopper)],
                        ['Training loss & MAE', history.history['loss'][min_idx], history.history['mean_absolute_error'][min_idx]  ], 
                        ['Validation loss & mae', history.history['val_loss'][min_idx], history.history['val_mean_absolute_error'][min_idx] ],
                       ], tablefmt="jira", numalign="right", floatfmt=".3f"))
            print('______________________________________________________________________')
        
        
        result = [{'model_name': m['name'], 'config': m, 'train_loss': history.history['loss'][min_idx], 'train_rmse': 0,
                   'train_mae': history.history['mean_absolute_error'][min_idx], 'train_mape': 0,
                   'valid_loss': history.history['val_loss'][min_idx], 'valid_rmse': 0, 
                   'valid_mae': history.history['val_mean_absolute_error'][min_idx],'valid_mape': 0, 
                   'test_rmse': 0, 'test_mae': 0, 'test_mape': 0, 'epochs': '{}/{}'.format(min_epoch, epochs), 'batch_train':m['batch_size'],
                   'input_shape':(X_train.shape[0], timesteps, X_train.shape[1]), 'total_time':t.time()-stopper, 
                   'time_step':0, 'dropout': m['layers'][0]['dropout']
                  }]
        # , 'splits':str(split_date)
        results = results.append(result, ignore_index=True)
        
        # Saving the model and weights
        model.save(model_dir + m['name'] + '.h5')
        
        # Write results to csv
        results.to_csv(output_table, sep=';')
        
        K.clear_session()
        import tensorflow as tf
        tf.reset_default_graph()
        
    # Shouldn't catch all errors, but for now...
    except BaseException as e:
        print('=============== ERROR {}/{} ============='.format(idx+1, len(models)))
        print(tabulate([['Model:', m['name']], ['Config:', m]], tablefmt="jira", numalign="right", floatfmt=".3f"))
        print('Error: {}'.format(e))
        result = [{'model_name': m['name'], 'config': m, 'train_loss': str(e)}]
        results = results.append(result, ignore_index=True)
        results.to_csv(output_table,sep=';')
        continue
        

========================= Model 1/108 =========================
| Starting with model | 06_1_l-1                   |
| Starting time       | 2018-07-03 10:54:51.786415 |


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


========================= Model 2/108 =========================
| Starting with model | 06_2_l-1_d-0.1             |
| Starting time       | 2018-07-03 10:54:59.031925 |


========================= Model 3/108 =========================
| Starting with model | 06_3_l-1_d-0.2             |
| Starting time       | 2018-07-03 10:55:05.838036 |


========================= Model 4/108 =========================
| Starting with model | 06_4_l-1_l-5               |
| Starting time       | 2018-07-03 10:55:18.289268 |


...

========================= Model 104/108 =========================
| Starting with model | 06_104_l-15_l-10_l-5_d-0.1 |
| Starting time       | 2018-07-03 11:32:58.754781 |


========================= Model 105/108 =========================
| Starting with model | 06_105_l-15_l-10_l-5_d-0.2 |
| Starting time       | 2018-07-03 11:33:32.431086 |


========================= Model 106/108 =========================
| Starting with model | 06_106_l-15_l-10_l-10      |
| Starting time       | 2018-07-03 11:34:07.942927 |


========================= Model 107/108 =========================
| Starting with model | 06_107_l-15_l-10_l-10_d-0.1 |
| Starting time       | 2018-07-03 11:34:39.980712  |


========================= Model 108/108 =========================
| Starting with model | 06_108_l-15_l-10_l-10_d-0.2 |
| Starting time       | 2018-07-03 11:35:14.063327  |


## Model selection based on the validation MAE

Select the top 5 models based on the Mean Absolute Error in the validation data:
http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error

In [9]:
# Number of the selected top models 
selection = 5
# If run in the same instance not necessary. If run on the same day, then just use output_table
results_fn = res_dir + model_cat_id + '_results_' + '20180703' + '.csv'

results_csv = pd.read_csv(results_fn, delimiter=';')
top_models = results_csv.nsmallest(selection, 'valid_mae')
print(top_models)

     Unnamed: 0                  model_name  \
67           67    06_68_l-10_l-5_l-5_d-0.1   
130         130    06_23_l-1_l-10_l-5_d-0.1   
104         104  06_105_l-15_l-10_l-5_d-0.2   
14           14     06_15_l-1_l-5_l-5_d-0.2   
159         159         06_52_l-5_l-10_l-10   

                                                config  dropout  train_loss  \
67   {'name': '06_68_l-10_l-5_l-5_d-0.1', 'layers':...      0.1    1.002538   
130  {'name': '06_23_l-1_l-10_l-5_d-0.1', 'layers':...      0.1    1.003049   
104  {'name': '06_105_l-15_l-10_l-5_d-0.2', 'layers...      0.2    1.003666   
14   {'name': '06_15_l-1_l-5_l-5_d-0.2', 'layers': ...      0.2    1.003120   
159  {'name': '06_52_l-5_l-10_l-10', 'layers': [{'t...      0.0    1.002655   

     train_rmse  train_mae  train_mape  valid_loss  valid_rmse   ...    \
67            0   0.905184           0    0.975022           0   ...     
130           0   0.905259           0    0.973894           0   ...     
104           0   0.

## Evaluate top 5 models

In [12]:
# Init test results table
test_results = pd.DataFrame(columns=['Model name', 'Mean absolute error', 'Mean squared error'])

# Init empty predictions
predictions = {}

# Loop through models
for index, row in top_models.iterrows():
    filename = model_dir + row['model_name'] + '.h5'
    model = load_model(filename)
    batch_size = int(row['batch_train'])
    
    
    # Calculate scores
    loss, mae = lstm.evaluate_model(model=model, X=X_test, y=y_test, batch_size=batch_size, timesteps=1, verbose=verbose)

    # Store results
    result = [{'Model name': row['model_name'], 
               'Mean squared error': loss, 'Mean absolute error': mae
              }]
    test_results = test_results.append(result, ignore_index=True)

    # Generate predictions
    model.reset_states()
    model_predictions = lstm.get_predictions(model=model, X=X_test, batch_size=batch_size, timesteps=timesteps[0], verbose=verbose)

    # Save predictions
    predictions[row['model_name']] = model_predictions

    K.clear_session()
    import tensorflow as tf
    tf.reset_default_graph()


test_results = test_results.sort_values('Mean absolute error', ascending=True)
test_results = test_results.set_index(['Model name'])

if not os.path.isfile(test_output_table):
    test_results.to_csv(test_output_table, sep=';')
else: # else it exists so append without writing the header
    test_results.to_csv(test_output_table,mode = 'a',header=False, sep=';')

In [13]:
print('Test dataset performance of the best {} (out of {} tested models):'.format(min(selection, len(models)), len(models)))
print(tabulate(test_results, headers='keys', tablefmt="grid", numalign="right", floatfmt=".3f"))

Test dataset performance of the best 5 (out of 108 tested models):
+----------------------------+-----------------------+----------------------+
| Model name                 |   Mean absolute error |   Mean squared error |
+============================+=======================+======================+
| 06_105_l-15_l-10_l-5_d-0.2 |                 0.912 |                1.007 |
+----------------------------+-----------------------+----------------------+
| 06_68_l-10_l-5_l-5_d-0.1   |                 0.912 |                1.007 |
+----------------------------+-----------------------+----------------------+
| 06_52_l-5_l-10_l-10        |                 0.912 |                1.007 |
+----------------------------+-----------------------+----------------------+
| 06_15_l-1_l-5_l-5_d-0.2    |                 0.912 |                1.008 |
+----------------------------+-----------------------+----------------------+
| 06_23_l-1_l-10_l-5_d-0.1   |                 0.912 |                1.008

In [15]:
restored_1=scalery.inverse_transform(model_predictions)
for i in range(len(restored_1)):
    print(restored_1[i])
# restored_2=scalery.inverse_transform(y_test)
# for i in range(len(restored_2)):
#             print(restored_2[i])

[305.86615]
[305.86865]
[305.86398]
[305.85342]
[305.86472]
[305.8601]
[305.86282]
[305.86057]
[305.4461]
[305.4411]
[305.43173]
[305.41656]
[305.4244]
[305.4322]
[305.43942]
[306.83002]
[304.82205]
[304.80103]
[304.7979]
[304.7839]
[304.78308]
[304.80713]
[306.19708]
[307.3917]
[304.2183]
[305.5149]
[304.20038]
[304.1874]
[304.18887]
[304.20132]
[307.72607]
[307.17725]
[305.0176]
[306.03906]
[303.7633]
[303.75742]
[303.76132]
[303.7526]
[308.7955]
[306.35382]
[306.74002]
[305.99518]
[303.524]
[303.52032]
[303.5313]
[303.50876]
[308.9071]
[305.2709]
[308.6962]
[305.48816]
[303.46533]
[304.5673]
[303.47235]
[303.4492]
[308.1305]
[304.24188]
[310.03598]
[306.20526]
[304.60458]
[306.4687]
[303.53717]
[303.51462]
[306.80075]
[303.46948]
[310.30328]
[306.64822]
[305.48642]
[307.8406]
[303.68127]
[303.6699]
[305.30765]
[303.0209]
[309.54407]
[306.44455]
[305.79483]
[308.2215]
[303.8618]
[303.87006]
[303.97867]
[302.86545]
[308.09448]
[307.21417]
[305.6414]
[307.70346]
[304.06036]
[304.08133]


[303.7649]
[305.10406]
[305.28253]
[305.18582]
[306.7627]
[304.7096]
[306.83698]
[304.25705]
[302.49823]
[305.1035]
[304.70065]
[305.9613]
[306.64822]
[305.2303]
[306.53815]
[304.416]
[301.71997]
[305.10693]
[304.13696]
[306.17493]
[306.08762]
[305.24747]
[306.03198]
[304.55933]
[301.4077]
[305.11246]
[303.72626]
[305.94162]
[305.3403]
[304.9818]
[305.4921]
[305.66116]
[302.7105]
[305.11353]
[303.5092]
[306.75827]
[304.6362]
[304.63266]
[305.03705]
[307.43628]
[303.8315]
[305.11627]
[303.45837]
[307.21802]
[304.10538]
[304.3371]
[304.71518]
[308.6497]
[304.39264]
[305.12207]
[303.5314]
[307.01996]
[303.78253]
[304.149]
[304.5243]
[309.93323]
[304.4743]
[305.1262]
[303.6846]
[306.31693]
[303.65228]
[304.06888]
[304.43954]
[310.54752]
[304.2876]
[305.1231]
[303.88144]
[305.416]
[303.67645]
[304.08347]
[304.42587]
[310.19733]
[304.02792]
[305.12326]
[305.09442]
[304.58362]
[303.79688]
[304.1623]
[304.46313]
[310.39038]
[303.8208]
[305.12143]
[305.97418]
[303.96545]
[304.98688]
[304.28055


[305.57584]
[306.06314]
[304.68958]
[303.87924]
[307.75284]
[307.4651]
[303.8074]
[306.80936]
[306.1222]
[305.12717]
[304.77823]
[303.88303]
[306.97824]
[306.76987]
[303.9123]
[305.7523]
[306.11182]
[304.26773]
[304.849]
[303.97034]
[305.8005]
[305.67737]
[304.05832]
[304.72348]
[305.67453]
[303.6308]
[305.87842]
[304.09973]
[304.573]
[304.43063]
[305.22034]
[303.92548]
[305.06583]
[304.51492]
[306.59503]
[304.24234]
[303.56924]
[303.2805]
[306.06113]
[303.42294]
[304.49612]
[305.22714]
[306.75305]
[304.39133]
[302.91113]
[302.42596]
[306.32172]
[304.38757]
[304.07382]
[305.38657]
[306.46774]
[305.51373]
[302.60614]
[301.95264]
[306.12152]
[305.16296]
[303.8289]
[305.1209]
[305.96646]
[306.3029]
[303.76614]
[301.83673]
[305.6814]
[306.70056]
[303.73526]
[304.6606]
[306.63153]
[306.52597]
[304.72086]
[301.98157]
[305.2042]
[308.5432]
[303.76978]
[304.2045]
[307.03064]
[307.52975]
[305.1212]
[302.3029]
[304.80276]
[310.29358]
[304.9208]
[303.86328]
[306.8701]
[308.99905]
[305.05554]
[30

[301.69888]
[303.8921]
[305.67368]
[303.1149]
[308.2238]
[310.49884]
[305.1081]
[303.19775]
[301.8022]
[303.5974]
[305.20004]
[303.5026]
[306.48453]
[309.4177]
[305.1131]
[303.24112]
[302.11295]
[304.64276]
[304.7962]
[303.85092]
[304.82224]
[307.7869]
[305.1156]
[303.38696]
[302.53806]
[305.43484]
[304.51907]
[305.13794]
[303.50748]
[305.965]
[305.12112]
[303.59164]
[302.99374]
[305.65118]
[304.36502]
[306.05887]
[302.65088]
[304.24936]
[305.11792]
[303.82635]
[303.42456]
[305.42032]
[304.31314]
[306.37393]
[302.23166]
[302.8808]
[305.10574]
[304.0624]
[304.7991]
[304.96072]
[304.32748]
[307.4446]
[302.16513]
[301.98312]
[305.102]
[304.28604]
[305.79736]
[304.48843]
[305.39703]
[308.02673]
[302.33792]
[301.56155]
[305.10077]
[304.48514]
[306.17184]
[304.12015]
[306.16904]
[307.8314]
[303.73853]
[301.53497]
[306.06885]
[304.65112]
[306.05994]
[303.9068]
[306.383]
[307.03302]
[304.84372]
[301.78223]
[306.73376]
[304.78]
[305.6914]
[303.83688]
[306.15073]
[305.9804]
[305.3401]
[302.18768


[306.55884]
[304.29974]
[305.10403]
[303.49118]
[304.33054]
[305.54398]
[303.4199]
[305.98035]
[306.35062]
[304.93002]
[306.07852]
[303.58643]
[304.49847]
[304.06256]
[303.27618]
[305.4549]
[305.90503]
[305.44653]
[306.75214]
[303.75085]
[304.6425]
[302.992]
[303.31256]
[304.936]
[305.40656]
[305.44382]
[306.87354]
[304.96484]
[304.7657]
[302.3695]
[304.5309]
[304.52072]
[304.9713]
[305.0271]
[306.56644]
[305.859]
[305.83273]
[302.1517]
[305.44507]
[304.2455]
[304.65277]
[304.42856]
[306.04398]
[306.16898]
[306.57724]
[302.2378]
[305.78403]
[304.1134]
[304.45572]
[303.8821]
[305.4956]
[306.0023]
[306.75806]
[303.62256]
[305.66608]
[304.0945]
[304.37317]
[303.4931]
[305.03073]
[305.58978]
[306.49313]
[304.72827]
[305.29904]
[304.1537]
[304.37015]
[303.30542]
[304.69653]
[305.13474]
[305.99313]
[306.4897]
[304.88406]
[304.25253]
[304.42563]
[303.29642]
[304.49292]
[304.74863]
[305.4603]
[307.59912]
[304.53696]
[304.37454]
[304.50604]
[303.41537]
[304.3968]
[304.4778]
[306.16418]
[307.75

...

[305.50168]
[308.20523]
[306.66348]
[305.53864]
[309.00208]
[301.6931]
[302.98355]
[304.9388]
[305.53845]
[307.4024]
[307.95532]
[306.1892]
[307.38867]
[301.5363]
[303.01218]
[304.6333]
[305.17105]
[306.24225]
[308.67606]
[306.31116]
[305.6882]
[301.69553]
[303.17]
[305.53186]
[304.64392]
[305.05963]
[308.52295]
[306.0014]
[305.76376]
[302.04947]
[303.39554]
[306.18546]
[304.13892]
[305.49527]
[307.68546]
[305.48788]
[305.86752]
[302.4983]
[304.68643]
[306.29318]
[303.77466]
[305.8368]
[306.52258]
[304.96286]
[305.52927]
[304.00668]
[305.64447]
[305.96994]
[303.58627]
[307.06204]
[305.3529]
[304.539]
[304.82654]
[305.15247]
[305.99393]
[306.73666]
[304.67358]
[307.82715]
[304.39517]
[304.26224]
[303.9968]
[305.66968]
[305.86264]
[307.18317]
[306.5862]
[307.74036]
[305.02945]
[304.12552]
[303.26688]
[305.66135]
[305.48495]
[306.98325]
[307.96353]
[306.92755]
[305.55887]
[304.09958]
[302.7728]
[305.36603]
[305.0562]
[306.29764]
[308.3384]
[305.7296]
[305.5923]
[304.15182]
[302.55374]
[30

[305.00885]
[305.02084]
[305.1184]
[305.63162]
[306.84067]
[310.00665]
[306.5548]
[304.6557]
[305.04123]
[305.0507]
[305.10983]
[306.2308]
[306.54227]
[310.04874]
[305.3716]
[303.45026]
[305.07184]
[305.06863]
[305.10986]
[306.2878]
[306.0283]
[309.3286]
[304.40622]
[304.0775]
[305.09756]
[305.08243]
[305.10684]
[305.91113]
[305.48917]
[307.98886]
[303.75156]
[304.69162]
[306.08395]
[305.09265]
[305.1087]
[305.3498]
[305.0332]
[306.32587]
[303.39655]
[304.83768]
[306.76364]
[305.10303]
[305.10953]
[304.80646]
[304.70157]
[304.65182]
[303.28958]
[304.57684]
[306.87695]
[305.1085]
[305.11588]
[304.38754]
[304.4961]
[303.2078]
[303.35468]
[304.12613]
[306.55518]
[305.11542]
[305.124]
[304.12653]
[304.40457]
[303.71918]
[304.5776]
[303.69287]
[307.2518]
[305.12463]
[305.13113]
[304.01132]
[304.39758]
[304.26917]
[306.5652]
[303.3929]
[308.61612]
[305.124]
[305.13593]
[305.07205]
[304.44254]
[304.36627]
[307.96304]
[303.26263]
[309.5186]
[305.12158]
[305.13846]
[305.84525]
[305.51413]
[304.

[303.08057]
[307.39905]
[302.78616]
[306.53183]
[306.53876]
[304.16672]
[304.38254]
[302.85403]
[303.13702]
[306.27127]
[302.09253]
[306.02322]
[307.2478]
[304.21762]
[304.3271]
[302.80035]
[303.32294]
[305.1122]
[303.13937]
[305.49213]
[308.61902]
[304.30798]
[305.36734]
[304.05063]
[303.56903]
[304.15775]
[304.07138]
[306.19284]
[310.16296]
[304.41626]
[307.15402]
[305.02045]
[303.83298]
[303.5109]
[304.4721]
[306.66208]
[311.1707]
[305.51733]
[308.3988]
[305.41135]
[304.08075]
[303.18164]
[304.4146]
[306.58878]
[311.17615]
[306.2987]
[309.7288]
[305.3373]
[304.2964]
[303.1149]
[304.1308]
[306.09808]
[310.21054]
[306.51404]
[311.04312]
[305.01474]
[304.48566]
[303.22372]
[303.82156]
[305.43478]
[310.04538]
[306.26834]
[311.7748]
[304.63937]
[304.64258]
[303.43314]
[303.6017]
[304.80496]
[309.61636]
[305.80206]
[311.52737]
[305.52142]
[304.77405]
[303.68246]
[303.5093]
[304.31934]
[308.58548]
[305.3037]
[310.36945]
[306.1456]
[305.84445]
[303.93973]
[303.53418]
[304.01797]
[307.10886]


[305.08652]
[304.9631]
[308.1131]
[308.04712]
[303.20724]
[305.7963]
[305.74683]
[304.30484]
[305.083]
[305.85147]
[306.86954]
[307.8204]
[303.09055]
[306.36777]
[306.03613]
[304.44034]
[305.08502]
[306.16122]
[305.47177]
[307.0259]
[304.26672]
[306.38965]
[305.85614]
[304.5708]
[305.09073]
[306.0065]
[304.23306]
[305.97995]
[306.27982]
[307.30743]
[305.4475]
[304.6825]
[305.10318]
[305.6138]
[304.74277]
[304.96268]
[307.7262]
[308.7665]
[304.99777]
[305.7515]
[305.11285]
[305.16595]
[305.19415]
[304.15665]
[308.16187]
[309.72473]
[304.627]
[306.50146]
[305.12015]
[305.9461]
[305.17026]
[303.63715]
[307.67322]
[309.71072]
[304.36588]
[306.67426]
[305.12732]
[306.481]
[306.22156]
[303.39407]
[308.10553]
[308.7999]
[304.21942]
[306.40717]
[305.1241]
[306.4564]
[306.8784]
[303.36255]
[308.2109]
[307.33487]
[304.17593]
[307.14856]
[305.1241]
[305.99753]
[308.09464]
[304.54425]
[308.9718]
[305.7094]
[304.20807]
[308.54752]
[306.0934]
[305.35434]
[308.74774]
[306.51797]
[309.28024]
[304.239

[302.08514]
[306.9758]
[301.8629]
[303.6227]
[304.99078]
[302.50162]
[304.19415]
[304.924]
[302.26715]
[306.3964]
[303.2251]
[303.96045]
[305.02765]
[303.7047]
[303.6631]
[306.7239]
[302.58936]
[305.42703]
[304.3392]
[304.24698]
[305.05838]
[305.81113]
[303.41068]
[308.0098]
[304.02673]
[304.31686]
[304.87805]
[304.4775]
[305.08237]
[307.36392]
[303.36578]
[308.31467]
[305.12323]
[303.3188]
[306.25757]
[304.65936]
[305.10358]
[307.89084]


In [24]:
# history.history['mean_absolute_error']
# history.history['val_mean_absolute_error']
# history.history['loss']
# history.history['val_loss']

[0.9752100993917413, 0.9752593324200748, 0.975902238733148]